**class activation map visulization by heatmaps(CAM)**


In [236]:
# loading the VGG16 network with pretrained weights

from keras.applications.vgg16 import VGG16

model = VGG16(weights = 'imagenet')

In [237]:
# preprocessing an input image for VGG16

from keras.preprocessing import image

from tensorflow.keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input
import numpy as np
from matplotlib import pyplot as plt


img_path = '../input/elephant/Elephants.jpeg'
img = image.load_img(img_path, target_size=(224, 224)) #resize to 224, 224
x = image.img_to_array(img) #float32 numpy array of shape
x = np.expand_dims(x, axis=0)#add dimension to transform the array into batch of size

x = preprocess_input(x)# preprocess the batch 

preds = model.predict(x)
print("Predicted:", decode_predictions(preds, top=3)[0])








In [238]:
input_img = x / 255
plt.imshow(input_img[0])
plt.show()

In [239]:
# maximally activated
np.argmax(preds[0])

In [240]:
# setting up the grand_cam algorithm
import cv2
import tensorflow as tf
from keras import backend as K

african_elephant_output_inx = 400


last_conv_layer = model.get_layer('block5_conv3')#the last convolutional layer in vgg16



In [241]:
from keras import backend as K
# Construct the gradient model by supplying:\n",
# (1) the inputs to our pre-trained model;\n",
# (2) the output of the last convolutional layer in the network;\n",
# (3) the output of the softmax activations from the model\n",

# grads = k.gradients(cat_dog_ouput_inx, last_conv_laye.output)[0]

grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])

In [242]:
# Set up GradientTape for automatic differentiation\n",
with tf.GradientTape() as tape:
    # pass the image through the gradient model, and grab the loss associated with the specific class index\n",
    conv_outputs, predictions = grad_model(x)
    loss = predictions[:, african_elephant_output_inx]

output = conv_outputs[0]

In [243]:
# Compute the gradients of the \"African elephant\" class with regard to the output \n",
# feature map of block5_conv3, given a sample image\n",
grads = tape.gradient(loss, conv_outputs)[0]

In [244]:
# Vector of shape (512), where each entry is the mean intensity of the gradient over a specific feature map channel\n",
weights = tf.reduce_mean(grads, axis=(0, 1))

In [245]:
# Multiplies each channel in feature-map array by \"how important this channel is\" with regard to the elephant class.\n",
cam = tf.multiply(weights, output)

# The channel-wise mean of the resulting feature map is the heatmap of the class activation!\n",
heatmap = tf.reduce_mean(cam, axis=-1).numpy()

In [246]:
#### Visualize the heatmap"
    
# Postprocessing heatmap\n",
heatmap = (heatmap - np.min(heatmap)) / (heatmap.max() - heatmap.min())
plt.matshow(heatmap)

In [247]:
### Generate superimposed original image over the heatmap"
# Use cv2 to load the original image\n",
cv_img = cv2.imread(img_path)

In [248]:
# Resize the heatmap to be the same as the original image\n",
heatmap = cv2.resize(heatmap, (cv_img.shape[1], cv_img.shape[0]))

In [249]:
# Converts heatmap to RBG\n",
heatmap = (heatmap * 255).astype(np.uint8)

In [250]:
# Apply a color map to the heatmap\n",
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

In [251]:
# Applies the heatmap to the original image. The variable alpha is a heatmap intensity factor\n",
alpha = 0.4
output_image = cv2.addWeighted(cv_img, alpha, heatmap, 1 - alpha, 0)



cv2.imwrite('../input/elephant/Elephants.jpeg', output_image)

plt.imshow(output_image)
plt.show()